In [1]:
# Required Code to run the notebook with a library & database on google drive

from google.colab import drive

folder="Twitter_analysis"

directory=f"/content/drive/MyDrive/{folder}/"
drive.mount("/content/drive/" ,force_remount=True)

## PopUp: Allow accessing google drive.

import sys
sys.path.append(directory+"myTwitterLibrary/")
import myTwitterLibrary

%load_ext autoreload
%autoreload 2

Mounted at /content/drive/


In [2]:
SearchTerm="(#AI (#sustainability OR #sustainable)) OR #SustainableAI OR #GreenAI" ### 2023-03-02
start_time='2010-01-01'
end_time='2023-03-30'
max_results_per_file=500
target_filename='SustainableAI'

# BToken from Twitter:


query=myTwitterLibrary.generate_query(SearchTerm,[start_time,end_time],max_results_per_file)

print(target_filename, query)

SustainableAI {'query': '(#AI (#sustainability OR #sustainable)) OR #SustainableAI OR #GreenAI', 'start_time': '2010-01-01T00:00:00.000000Z', 'end_time': '2023-03-30T00:00:00.000000Z', 'max_results': 500, 'tweet.fields': 'author_id,created_at,entities,lang,                public_metrics,in_reply_to_user_id,referenced_tweets,conversation_id,attachments', 'user.fields': 'created_at,description,entities,id,location,name,username', 'media.fields': 'type,alt_text,url,height,preview_image_url', 'expansions': 'attachments.media_keys,referenced_tweets.id,referenced_tweets.id.author_id,                entities.mentions.username,in_reply_to_user_id'}


In [3]:
# TEST Twitter API --> Response 403 means no authorisation. 401 means "no valid token"; Twitter KAPUTT!

import requests
search_url = "https://api.twitter.com/2/tweets/counts/recent"
headers=myTwitterLibrary.create_headers(BToken)
params=query

requests.request("GET", search_url, headers=headers, params=params)

<Response [403]>

In [5]:
#key_file=directory+"Twitter_API_key.json"

myTwitterLibrary.run_search(query, target_filename=target_filename, BToken=BToken, Max_files=15, Max_retry=2, sleep_time=0.5,key_file=False)


Trial number:  0
0 0
403
retry :D :D :D 0 0
Trial number:  1
0 0
403
retry :D :D :D 1 0


In [6]:
import FileImport
#reaload the library

combined_results,combined_media,combined_user,combined_extTweets=FileImport.FileImport(directory,target_filename)

df=FileImport.prepare_df(combined_results,combined_media,combined_user, Simplify=True)


len: 5 ; /content/drive/MyDrive/Twitter_analysis/2023-05-08_M_SustainableAI__0_data.json, /content/drive/MyDrive/Twitter_analysis/2023-05-08_M_SustainableAI__1_data.json, /content/drive/MyDrive/Twitter_analysis/2023-05-08_M_SustainableAI__2_data.json, /content/drive/MyDrive/Twitter_analysis/2023-05-08_M_SustainableAI__3_data.json, /content/drive/MyDrive/Twitter_analysis/2023-05-08_M_SustainableAI__4_data.json
Tweets: 2367, Media:420, Users: 1686


In [ ]:
!pip install spacytextblob
!pip install tweet-preprocessor
!pip install tqdm

In [21]:
import nlp

%load_ext autoreload
%autoreload 2

df=nlp.NLP_Pipeline(df, language="en")
df.to_json(directory+FileImport.Today+target_filename+"_all.json")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cleaning done.
Token & Lemmatizing & stopword removal & modal_word.


100%|██████████| 2367/2367 [00:31<00:00, 75.23it/s]

Current Time = 17:40:07


In [22]:
from transformers import pipeline
import nlp

model="cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_pipe = pipeline("sentiment-analysis", model=model, max_length=512, truncation=True)

print("Stopwording done. Next: sentiment.")
df["sentiment"] = df.progress_apply(nlp.roberta_sentiment, axis=1,column="clean_text",pipe=sentiment_pipe)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Stopwording done. Next: sentiment.


100%|██████████| 2367/2367 [07:11<00:00,  5.49it/s]


In [24]:
df.to_json(directory+FileImport.Today+"_"+target_filename+"_all.json")